# **Smart Resume Matching**
### **Azure SQL DB 기반 RAG를 통한 유사성 검색**

-----

*

본 튜토리얼은 [Microsoft Dev-Blog: Smart Resume Matching with Azure SQL DB and Document Intelligence](https://devblogs.microsoft.com/azure-sql/smart-resume-matching-with-azure-sql-db-document-intelligence/)를 참고하여 한국어로 작성됐습니다.

현재 Microsoft를 비롯한 여러 회사에는 특정 직무에 적합한 이력서를 추천받거나 쉽게 키워드 검색을 할 수 있는 Smart Resume 서비스가 도입돼 있습니다. 본 튜토리얼은 Azure의 여러 서비스를 활용해 Smart Resume Matching을 직접 구현하는 것을 목표로 합니다. 이를 위해 Azure AI Services 중 Document Intelligence를 통한 PDF chunking, Azure OpenAI를 이용한 텍스트 임베딩, Azure SQL DB의 vector data type을 이용한 임베딩 저장 및 vector 유사도 검색과 RAG를 수행합니다.

**기존 블로그의 내용에 더해, Azure OpenAI를 통해 Sample Dataset을 형성하는 과정과 API Management (APIM) 리소스를 도입해 로드밸런싱을 하는 과정을 추가했습니다.**

*

-----

### Tutorial Overview

다음의 과정을 따라 Smart Resume Matching을 구현합니다.

1. Resume PDF Sample Data 생성 (with Azure OpenAI chat completion model)
2. PDF Chunking (with Azure Document Intelligence)
3. 임베딩 생성 (with Azure OpenAI text embdding model)
4. 벡터 저장 및 유사도 검색 (with Azure SQL DB built-in vector functions)
5. RAG 기반 답변 생성 (with Azure OpenAI chat completion model)

-----

### Prerequisites

1. Azure Subscription
2. Azure Resources: Azure Document Intelligence, Azure SQL Database, Azure OpenAI (Chat Completion Model, Embedding Model), Azure API Management
3. .env: 아래의 셀을 실행해 작성할 수 있습니다.
4. Python: 본 튜토리얼은 3.10.11 버전으로 테스트 됐습니다.
5. Jupyter Notebook: 본 튜토리얼은 Azure Machine Learning Studio의 Notebook (Python 3.10 - SDK v2) 환경에서 작성 및 테스트됐습니다.

**구체적인 튜토리얼: [Microsoft Korea 테크블로그 (TBU)] (link_tbu)**


In [ ]:
%%writefile .env

APIM_URL = ""
APIM_SUBSCRIPTION_KEY = ""
APIM_API_VERSION = ""
APIM_CHAT_DEPLOYMENT_NAME = ""
APIM_EMBEDDING_DEPLOYMENT_NAME = ""

AZUREDOCINTELLIGENCE_ENDPOINT = ""
AZUREDOCINTELLIGENCE_API_KEY = ""

# Use only one of the below. The one you are not using should be commented out.

# For Entra ID Service Principle Authentication: Paste Connection String {ODBC (Includes Node.js) (Microsoft Entra password authentication)} and change 18 to 17
ENTRA_CONNECTION_STRING= ""

# For SQL Authentication: Paste Connection String {ODBC (Includes Node.js) (SQL authentication)} and change Dirver 18 to 17
SQL_CONNECTION_STRING= ""

# 환경설정

In [1]:
#Setup the python libraries required for this notebook
import sys
!{sys.executable} -m pip install -r requirements.txt

In [19]:
#Load the env details
from dotenv import load_dotenv
load_dotenv()

True

# **PART 0: AOAI를 이용한 Sample Data 생성**

In [5]:
import os
import requests

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
import markdown2

In [56]:
apim_url = os.getenv("APIM_URL")
deployment_name = os.getenv("APIM_CHAT_DEPLOYMENT_NAME")
api_version = os.getenv("APIM_API_VERSION")
subscription_key = os.getenv("APIM_SUBSCRIPTION_KEY")

# Construct the URL and headers

chat_url = f"{apim_url}/deployments/{deployment_name}/chat/completions?api-version={api_version}"
headers = {
    "Content-Type": "application/json",
    "Ocp-Apim-Subscription-Key": subscription_key
}

In [18]:
folder_resumes_dir = 'resumes'

# Check if the folder exists
if not os.path.exists(folder_resumes_dir):
    os.makedirs(folder_resumes_dir)
    print(f"Folder '{folder_resumes_dir}' created.")
else:
    print(f"Folder '{folder_resumes_dir}' already exists.")

Folder 'resumes' already exists.


In [16]:
# Create n resumes as sample data

n_resumes = 100

for i in range(n_resumes):
    json_payload = {
        "messages": [
            {
                "role": "system",
                "content": "You are a job seeker with some academic or work experience in exactly in one of the fields: software dev, embedded systems, data science, NLP, computer vision, or IT consulting. Create a half-page length resume with a random name, random degree, and work experiences only relevant to the previously selected field."
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    response = requests.post(chat_url, headers=headers, json=json_payload)
    result = response.json()['choices'][0]['message']['content']

    # Save result as PDF
    pdf_file = f'{folder_resumes_dir}/resume_{i}.pdf'
    # Create a PDF file
    doc = SimpleDocTemplate(pdf_file, pagesize=letter,
                      rightMargin=20, leftMargin=20,
                      topMargin=20, bottomMargin=20)
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Smaller', fontSize=7))
    story = []

    # Convert markdown to HTML
    html_content = markdown2.markdown(result)

    # Add the HTML content to the PDF
    for line in html_content.split('\n'):
        story.append(Paragraph(line, styles['Smaller']))
        story.append(Spacer(1, 2))
        
    doc.build(story)

# **PART 1: Azure Document Intelligence를 이용한 PDF Extraction 및 Tokenization**
-----
Azure Document Intelligence를 사용하면 사전 정의된 모델로 문서를 분석할 수 있습니다. 본 튜토리얼에서 DocumentAnalysisClient는 그 중 [layout 모델](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/prebuilt/layout?view=doc-intel-4.0.0&tabs=rest%2Csample-code)을 사용해 이력서 PDF 파일에서 텍스트를 추출합니다. 텍스트는 tiktoken 라이브러리를 통해 인코딩돼 토큰들로 변환됩니다. 이때 하나의 단어는 하나 이상의 토큰으로 변환될 수 있습니다.

다음 단계인 임베딩에서 텍스트가 Azure OpenAI 임베딩 모델의 처리 한도를 초과하지 않도록, 토큰들은 하나 이상의 chunk로 분할됩니다. Chunk는 500개 토큰 단위로 설정돼있습니다.

In [ ]:
import os
import re
import tiktoken
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

In [6]:
# Load environment variables
endpoint = os.getenv("AZUREDOCINTELLIGENCE_ENDPOINT")
api_key = os.getenv("AZUREDOCINTELLIGENCE_API_KEY")

# Create a DocumentAnalysisClient
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key)
)


In [10]:
# folder_resumes_dir: Path to the directory containing PDF files, set earlier

def get_pdf_files(folder_resumes_dir):
    for path, subdirs, files in os.walk(folder_resumes_dir):
        for name in files:
            if (name.endswith(".pdf")):
                yield os.path.join(path, name)

# Function to read PDF files and extract text using Azure AI Document Intelligence
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as f:
        poller = document_analysis_client.begin_analyze_document("prebuilt-layout", document=f)
    result = poller.result()
    text = ""
    for page in result.pages:
        for line in page.lines:
            text += line.content + " "
    return text

# Function to clean text and remove special characters
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    return text

# Function to split text into chunks of 500 tokens
def split_text_into_token_chunks(text, max_tokens=500):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(text)
    chunks = []
    
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
    
    return chunks

# Count the number of PDF files in the directory
pdf_files = [f for f in get_pdf_files(folder_resumes_dir)]
num_files = len(pdf_files)
print(f"Number of PDF files in the directory: {num_files}\n")

# Create a DataFrame to store the chunks
data = []

for file_id, pdf_file in enumerate(pdf_files):
    file_name = os.path.basename(pdf_file)

    pdf_path = pdf_file
    text = extract_text_from_pdf(pdf_path)
    cleaned_text = clean_text(text)
    chunks = split_text_into_token_chunks(cleaned_text)
    
    print(f"{file_name}... {len(chunks)} chunks")
    
    for chunk_id, chunk in enumerate(chunks):
        chunk_text = chunk.strip() if chunk.strip() else "NULL"
        unique_chunk_id = f"{file_id}_{chunk_id}"
        print(f"{file_name}... Chunk ID: {chunk_id}, Unique Chunk ID: {unique_chunk_id}, Chunk Length: {len(chunk_text)}, Chunk Text: {chunk_text[:20]}...\n")  # Print first 20 characters of chunk text
        data.append({
            "file_name": file_name,
            "chunk_id": chunk_id,
            "chunk_text": chunk_text,
            "unique_chunk_id": unique_chunk_id
        })

df = pd.DataFrame(data)
df.head(3)


Number of PDF files in the directory: 100

resume_0.pdf... 1 chunks
resume_0.pdf... Chunk ID: 0, Unique Chunk ID: 0_0, Chunk Length: 1890, Chunk Text: John Doe Education B...

resume_1.pdf... 2 chunks
resume_1.pdf... Chunk ID: 0, Unique Chunk ID: 1_0, Chunk Length: 2985, Chunk Text: Daniel Thompson dsth...

resume_1.pdf... Chunk ID: 1, Unique Chunk ID: 1_1, Chunk Length: 95, Chunk Text: 2020  Present  Techn...

resume_10.pdf... 1 chunks
resume_10.pdf... Chunk ID: 0, Unique Chunk ID: 2_0, Chunk Length: 2589, Chunk Text: John Smith Email Add...

resume_11.pdf... 1 chunks
resume_11.pdf... Chunk ID: 0, Unique Chunk ID: 3_0, Chunk Length: 3054, Chunk Text: Jane Doe New York NY...

resume_12.pdf... 1 chunks
resume_12.pdf... Chunk ID: 0, Unique Chunk ID: 4_0, Chunk Length: 1794, Chunk Text: John Doe johndoeexam...

resume_13.pdf... 1 chunks
resume_13.pdf... Chunk ID: 0, Unique Chunk ID: 5_0, Chunk Length: 2609, Chunk Text: Ella Johnson Email e...

resume_14.pdf... 1 chunks
resume_14.pdf... Ch

,file_name,chunk_id,chunk_text,unique_chunk_id
0,resume_0.pdf,0,John Doe Education Bachelor of Science in Comp...,0_0
1,resume_1.pdf,0,Daniel Thompson dsthompsonexamplecom 123 4567...,1_0
2,resume_1.pdf,1,2020 Present Technical Volunteer DataKind NY...,1_1


(Optional) 아래의 셀을 실행해 기존의 단어 개수와 최종 토큰 개수를 비교해봅시다.

In [12]:
# Compare number of tokens to number of words

import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")
sample_encode = tokenizer.encode(df.chunk_text[0])
words = df.chunk_text[0].count(" ")+1
decode = tokenizer.decode_tokens_bytes(sample_encode)
decoded_result = ', '.join([token.decode('utf-8') for token in decode])

print(df.chunk_text[0])
print(f"\nOriginally {words} words\nMapped to {len(decode)} tokens\n")
print(decoded_result)

John Doe Education Bachelor of Science in Computer Science University of California Berkeley Graduation May 2023 Experience Software Engineer Intern Tech Solutions Inc  San Francisco CA June 2022  August 2022  Developed and optimized frontend features using React reducing page load time by 25 and improving user experience  Collaborated with backend engineers to integrate APIs ensuring seamless data retrieval and user interaction  Conducted code reviews and unit testing to maintain code quality and reliability Junior Software Developer Innovex Systems  Los Angeles CA September 2021  May 2022  Participated in the full software development lifecycle for a web application aimed at enhancing digital collaboration  Implemented RESTful APIs using Nodejs improving data handling efficiency  Identified and fixed bugs in existing software resulting in a 15 increase in application stability Programming Tutor University of California Berkeley  Berkeley CA September 2019  May 2023  Provided oneonone

# **PART 2 : Azure Open AI를 이용한 임베딩 생성**

토큰화의 결과인 각 chunk에 대한 임베딩을 생성하는 과정입니다.

임베딩은 단어의 의미 관계를 고려한 값으로, 이후 유사도 검색을 할 수 있는 기반이 됩니다.

In [ ]:
import os
import re
import requests
from num2words import num2words
import pandas as pd
import numpy as np
import json
from openai import AzureOpenAI

In [ ]:
apim_url = os.getenv("APIM_URL")
deployment_name = os.getenv("APIM_EMBEDDING_DEPLOYMENT_NAME")
api_version = os.getenv("APIM_API_VERSION")
subscription_key = os.getenv("APIM_SUBSCRIPTION_KEY")

# Construct the URL and headers

embedding_url = f"{apim_url}/deployments/{deployment_name}/embeddings?api-version={api_version}"
headers = {
    "Content-Type": "application/json",
    "Ocp-Apim-Subscription-Key": subscription_key
}

In [13]:
def get_embedding(text):
    """
    Get sentence embedding using the Azure OpenAI text-embedding-small model.

    Args:
        text (str): Text to embed.

    Returns:
        list: A list containing the embedding.
    """

    response = requests.post(embedding_url, headers=headers, json={"input": [text]})
    # Embed the extracted chunk as text
    
    if response.status_code == 200:
        response_json = response.json()
        embedding = json.loads(str(response_json['data'][0]['embedding']))
        return embedding
    else:
        return None

# Example usage
all_filenames = []
all_chunkids = []
all_chunks = []
all_embeddings = []

# Assuming df is already defined with the required columns
for index, row in df.iterrows():
    filename = row['file_name']
    chunkid = row['unique_chunk_id']
    chunk = row['chunk_text']
    embedding = get_embedding(chunk)
    
    if embedding is not None:
        all_filenames.append(filename)
        all_chunkids.append(chunkid)
        all_chunks.append(chunk)
        all_embeddings.append(embedding)
    
    if (index + 1) % 50 == 0:  # Print progress every 50 rows
        print(f"Completed {index + 1} rows")

# Create a new DataFrame with the results
result_df = pd.DataFrame({
    'filename': all_filenames,
    'chunkid': all_chunkids,
    'chunk': all_chunks,
    'embedding': all_embeddings
})

result_df.head(5)  # Display the first few rows of the dataframe


Completed 50 rows
Completed 100 rows


,filename,chunkid,chunk,embedding
0,resume_0.pdf,0_0,John Doe Education Bachelor of Science in Comp...,"[-0.011703418, -0.013349316, 0.0018044173, -0...."
1,resume_1.pdf,1_0,Daniel Thompson dsthompsonexamplecom 123 4567...,"[-0.048026104, 0.00015640476, 0.03146353, -0.0..."
2,resume_1.pdf,1_1,2020 Present Technical Volunteer DataKind NY...,"[-0.018442072, -0.034961402, 0.0030465978, -0...."
3,resume_10.pdf,2_0,John Smith Email Address Phone Number Linked...,"[-0.0012448077, -0.00021796567, -0.004634464, ..."
4,resume_11.pdf,3_0,Jane Doe New York NY Email janedoeexamplecom P...,"[-0.022091128, -0.0047286567, 0.015094948, -0...."


# **PART 3 : Azure SQL DB를 사용한 vector 저장**

### **SQL DB 연결**

In [ ]:
import os
import pyodbc
import struct
import pandas as pd
from prettytable import PrettyTable
from azure.identity import DefaultAzureCredential

In [21]:
#lets define a function to connect to SQLDB

def get_mssql_connection():
    # Retrieve the connection string from the environment variables
    entra_connection_string = os.getenv('ENTRA_CONNECTION_STRING')
    sql_connection_string = os.getenv('SQL_CONNECTION_STRING')
    
    # Determine the authentication method and connect to the database
    if entra_connection_string:
        # Entra ID Service Principal Authentication
        credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)    
        token = credential.get_token('https://database.windows.net/.default')
        token_bytes = token.token.encode('UTF-16LE')
        token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)
        SQL_COPT_SS_ACCESS_TOKEN = 1256  # This connection option is defined by Microsoft in msodbcsql.h
        conn = pyodbc.connect(entra_connection_string, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct})
    
    elif sql_connection_string:
        # SQL Authentication
        conn = pyodbc.connect(sql_connection_string)
        
    else:
        raise ValueError("No valid connection string found in the environment variables.")

    return conn

### **임베딩을 vector data type으로 저장하기**

In [25]:
# Retrieve the connection string from the function get_mssql_connection()
conn = get_mssql_connection()

try:
    print(f"Success: {conn}")
except:
    print("Failed sql connection.")


# Create a cursor object
cursor = conn.cursor()

# Create table
create_table_query = """
IF OBJECT_ID('dbo.resumedocs', 'U') IS NOT NULL
DROP TABLE dbo.resumedocs

CREATE TABLE resumedocs (
        chunkid NVARCHAR(MAX),
        filename NVARCHAR(255),
        chunk NVARCHAR(MAX),
        embedding VECTOR(1536)
    )
"""
cursor.execute(create_table_query)
conn.commit()


Success: <pyodbc.Connection object at 0x7f1216fe1550>


In [26]:
# Enable fast_executemany
cursor.fast_executemany = True

# Loop through the DataFrame rows and insert them into the table
for index, row in result_df.iterrows():
    chunkid = row['chunkid']
    filename = row['filename']
    chunk = row['chunk']
    embedding = row['embedding']
    
    # Use placeholders for the parameters in the SQL query
    query = f"""
    INSERT INTO resumedocs (chunkid, filename, chunk, embedding)
    VALUES (?, ?, ?, CAST(CAST(? as NVARCHAR(MAX)) AS VECTOR(1536)))
    """
    # Execute the query with the parameters
    cursor.execute(query, chunkid, filename, chunk, json.dumps(embedding))

# Commit the changes
conn.commit()

# Print a success message
print("Data inserted successfully into the 'resumedocs' table.")

# Close the connection
conn.close()


Data inserted successfully into the 'resumedocs' table.


생성되고 텍스트 임베딩이 저장된 SQL 테이블의 형태는 아래와 같습니다.

In [27]:
# Retrieve the connection string from the environment variables
conn = get_mssql_connection()

# Create a cursor object
cursor = conn.cursor()

# Use placeholders for the parameters in the SQL query
query = "SELECT TOP(10) filename, chunkid, chunk, CAST(embedding AS NVARCHAR(MAX)) as embedding FROM dbo.resumedocs ORDER BY chunkid"

# Execute the query with the parameters
cursor.execute(query)
queryresults = cursor.fetchall()

# Get column names from cursor.description
column_names = [column[0] for column in cursor.description]

# Create a PrettyTable object
table = PrettyTable()

# Add column names to the table
table.field_names = column_names

# Set max width for each column to truncate data
table.max_width = 20

# Add rows to the table
for row in queryresults:
    # Truncate each value to 20 characters
    truncated_row = [str(value)[:20] for value in row]
    table.add_row(truncated_row)

# Print the table
print(table)

# Commit the changes
conn.commit()
# Close the connection
conn.close()


+---------------+---------+----------------------+----------------------+
|    filename   | chunkid |        chunk         |      embedding       |
+---------------+---------+----------------------+----------------------+
|  resume_0.pdf |   0_0   | John Doe Education B | [-1.1703418e-002,-1. |
|  resume_1.pdf |   1_0   | Daniel Thompson dsth | [-4.8026104e-002,1.5 |
|  resume_1.pdf |   1_1   | 2020  Present  Techn | [-1.8442072e-002,-3. |
| resume_18.pdf |   10_0  | Christina Taylor Con | [5.2000964e-003,-1.1 |
| resume_19.pdf |   11_0  | Akira Tanaka Contact | [-1.0703366e-002,-6. |
|  resume_2.pdf |   12_0  | John Doe 1234 Elm St | [-1.3231047e-002,1.6 |
| resume_20.pdf |   13_0  | Name Johnathan Doe E | [-1.4172998e-002,5.7 |
| resume_21.pdf |   14_0  | Alexander Johnson al | [-4.7440995e-003,-1. |
| resume_22.pdf |   15_0  | James Reynolds Conta | [-8.5942987e-003,-6. |
| resume_23.pdf |   16_0  | Johnathan Smith Emai | [-1.2715508e-002,1.1 |
+---------------+---------+-----------

### **VECTOR\_DISTANCE 함수를 이용해 벡터 유사도 검색하기**


이제 주어진 쿼리와의 벡터 유사도가 높은 파일을 검색하는 작업을 수행할 수 있습니다.

VECTOR\_DISTANCE 함수는 쿼리의 벡터와 SQL DB에 저장된 임베딩 간의 거리를 계산합니다. 거리 계산 방식이 되는 metric으로는 cosine, euclidean, dot을 선택할 수 있습니다.

In [28]:
import os
import pyodbc
import json
from dotenv import load_dotenv

def vector_search_sql(query, num_results=5):
    # Load environment variables from .env file
    load_dotenv()

    # Use the get_mssql_connection function to get the connection string details
    conn = get_mssql_connection()

    # Create a cursor object
    cursor = conn.cursor()

    # Generate the query embedding for the user's search query
    user_query_embedding = get_embedding(query)
    
    # SQL query for similarity search using the function vector_distance to calculate cosine similarity
    sql_similarity_search = f"""
    SELECT TOP(?) filename, chunkid, chunk,
           1-vector_distance('cosine', CAST(CAST(? as NVARCHAR(MAX)) AS VECTOR(1536)), embedding) AS similarity_score,
           vector_distance('cosine', CAST(CAST(? as NVARCHAR(MAX)) AS VECTOR(1536)), embedding) AS distance_score
    FROM dbo.resumedocs
    ORDER BY distance_score 
    """

    cursor.execute(sql_similarity_search, num_results, json.dumps(user_query_embedding), json.dumps(user_query_embedding))
    results = cursor.fetchall()

    # Close the database connection
    conn.close()

    return results
    
#example usage
vector_search_sql("database administrator", num_results=3)

[('resume_84.pdf', '83_1', 'contribute to the success of your organization', 0.7804182655475509, 0.2195817344524491),
 ('resume_93.pdf', '93_0', 'Resume Name John Smith Phone 555 1234567 Email johnsmithexamplecom LinkedIn linkedincominjohnsmith Objective Highly skilled and motivated software developer with a strong foundation in computer science and extensive experience in developing testing and maintaining software applications Proficient in multiple programming languages with a keen interest in producing highquality code Education Bachelor of Science in Computer Science University of TechVille Graduated May 2018 Skills  Programming Languages Java Python C  Web Development HTML CSS JavaScript Reactjs  Frameworks Spring Boot Django Nodejs  Databases MySQL PostgreSQL MongoDB  Tools Git Docker Jenkins  Cloud Services AWS Azure  Methodologies Agile Scrum Professional Experience Software Developer ABC Technologies New York NY June 2018  Present Designed developed and implemented scalable w

# **Part 4 : SQL DB 검색 결과를 이용해 LLM 성능 강화하기 (RAG)**

[Retrieval-Augmented Generation (RAG)](https://learn.microsoft.com/en-us/azure/search/retrieval-augmented-generation-overview)는 외부 지식 검색을 통해 AI의 성능을 강화하는 방법입니다.

이때 Retrieval은, 앞서 SQL DB에 저장해둔 벡터 임베딩의 쿼리 결과, 즉 높은 유사성의 지원자 정보라고 할 수 있습니다.

이 결과를 바탕으로 Azure Open AI는 유저에게 보다 정확한 답변을 생성할 수 있게 됩니다.

In [ ]:
apim_url = os.getenv("APIM_URL")
deployment_name = os.getenv("APIM_CHAT_DEPLOYMENT_NAME")
api_version = os.getenv("APIM_CHAT_DEPLOYMENT_VERSION")
subscription_key = os.getenv("APIM_SUBSCRIPTION_KEY")

# Construct the URL and headers

chat_url = f"{apim_url}/deployments/{deployment_name}/chat/completions?api-version={api_version}"
headers = {
    "Content-Type": "application/json",
    "Ocp-Apim-Subscription-Key": subscription_key
}

In [56]:
def generate_completion(search_results, user_input):
    system_prompt = '''
    You are an intelligent & funny assistant who will exclusively answer based on the data provided in the `search_results`:
    - Use the information from `search_results` to generate your top 3 responses. If the data is not a perfect match for the user's query, use your best judgment to provide helpful suggestions and include the following format:
    File: {filename}
    Chunk ID: {chunkid}
    Similarity Score: {similarity_score}
    Add a small snippet from the Relevant Text: {chunktext}
    Do not use the entire chunk
    - Avoid any other external data sources.
    - Add a summary about why the candidate maybe a goodfit even if exact skills and the role being hired for are not matching , at the end of the recommendations. Ensure you call out which skills match the description and which ones are missing. If the candidate doesnt have prior experience for the hiring role which we may need to pay extra attention to during the interview process.
    - Add a Microsoft related interesting fact about the technology that was searched 
    '''

    messages = [{"role": "system", "content": system_prompt}]
    
    # Create an empty list to store the results
    result_list = []

    # Iterate through the search results and append relevant information to the list
    for result in search_results:
        filename = result[0]  # Assuming filename is the first column
        chunkid = result[1]
        chunktext = result[2]
        similarity_score = result[3]  # Assuming similarity_score is the third column

        # Convert search results into a formatted text block
        result_text = "\n".join([
            f"File: {filename}\nChunk ID: {chunkid}\nSimilarity Score: {similarity_score}\nRelevant Text: {chunktext}"
            for result in search_results
        ])

    
    messages.append({"role": "user", "content": result_text})
    messages.append({"role": "user", "content": user_input})
    json_payload = {"messages": messages}  # Wrap messages inside a dictionary

    response = requests.post(chat_url, headers=headers, json=json_payload)

    return response.json()['choices'][0]['message']['content']
    

In [57]:
# Create a loop of user input and model output to perform Q&A on the PDF's that are now chunked and stored in the SQL DB with embeddings
#
# PLEASE NOTE: An input box will be displayed for the user to enter a question/query at the top of the scree.
# The model will then provide a response based on the data stored in the SQL DB.
# Type 'end' to end the session.
#

print("*** What Role are you hiring for? And What skills are you looking for? Ask me & I can help you find a candidate :) Type 'end' to end the session.\n")

while True:
    user_input = input("User prompt: ")
    if user_input.lower() == "end":
        break

    # Print the user's question
    print(f"\nUser asked: {user_input}")

  
    # Assuming vector_search_sql and generate_completion are defined functions that work correctly
    search_results = vector_search_sql(user_input)
    completions_results = generate_completion(search_results, user_input)

    # Print the model's response
    print("\nSmart matching results:")
    print(completions_results)

# The loop will continue until the user types 'end'

*** What Role are you hiring for? And What skills are you looking for? Ask me & I can help you find a candidate :) Type 'end' to end the session.


User asked: project manager

Smart matching results:
Here are the top 3 relevant chunks from the search results regarding a candidate suitable for a Project Manager role:

1. 
File: resume_6.pdf
Chunk ID: 56_0
Similarity Score: 0.7948388558350712
Relevant Text: 
"Participated in daily standups and contributed to sprint planning leading to timely project delivery Wrote unit tests and integrated testing procedures maintaining high code quality standards Collaborated with the UIUX team to improve the user interface and experience for customerfacing applications."

2. 
File: resume_6.pdf
Chunk ID: 56_0
Similarity Score: 0.7948388558350712
Relevant Text: 
"Lead the design and development of various modules using Java and Python Collaborated with crossfunctional teams to define software specifications and architecture Implemented RESTful APIs tha